In [ ]:
import json
import math
import os
from pprint import pprint

import numpy as np
import pandas as pd
import scipy.stats as statst


In [ ]:
# create a function that reads a file in and insert on the first line the following string objects: and save the file again
def insert_string_in_file(file_path, string):
    with open(file_path, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(string.rstrip('\r\n') + '\n' + content)

def insert_comma_after_closing_bracket(file_path):
    with open(file_path, 'r+') as f:
        content = f.read()
        content = content.replace('}', '},')
        f.seek(0, 0)
        f.write(content)

def truncate_last_char(file_path):
    with open(file_path, 'rb+') as f:
        f.seek(-1, os.SEEK_END)
        f.truncate()

def insert_string_in_file2(file_path, string):
    with open(file_path, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(content.rstrip('\r\n') + '\n' + string.rstrip('\r\n') + '\n')

def transform_file_to_json(fpath):
    insert_string_in_file(fpath, '{"objects": [')
    insert_comma_after_closing_bracket(fpath)
    truncate_last_char(fpath)
    insert_string_in_file2(fpath, ']}')


In [ ]:
fpath = os.path.join(os.getenv('HOME'), "Corpora/CLEVR-extracted/scenes/CLEVR_val_000001.json")
transform_file_to_json(fpath)

In [ ]:
base_dir = os.path.join(os.getenv('HOME'), "Corpora/CLEVR-extracted/scenes/")
input_dir = os.path.join(base_dir, "val")
output_dir = os.path.join(base_dir, "t-val")

for root, dirs, files in os.walk(input_dir):
    for fname in files:
        # if fname is not .DS_Store
        if (not fname.startswith('.')):
            print(fname)
            transform_file_to_json(os.path.join(root, fname))

In [ ]:
def find_extrema(scene, extrema):
    for obj in scene['objects']:
        for attr in obj:
            if attr in extrema:
                if obj[attr] < extrema[attr]['min']:
                    extrema[attr]['min'] = obj[attr]
                if obj[attr] > extrema[attr]['max']:
                    extrema[attr]['max'] = obj[attr]
            else:
                extrema[attr] = {'min': obj[attr], 'max': obj[attr]}
    return extrema

In [ ]:
base_dir = os.path.join(os.getenv('HOME'), "Corpora/CLEVR-extracted/scenes/")
input_dir = os.path.join(base_dir, "val")
output_dir = os.path.join(base_dir, "t-val")

extrema = {}
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        # if fname is not .DS_Store
        if (not fname.startswith('.')):
            with open(os.path.join(root, fname)) as f:
                scene = json.load(f)
                extrema = find_extrema(scene, extrema)

In [98]:
def normalise_val_in_range(x, a, b):
    return (x - a) / (b - a)

def normalise_val(attr, val):
    match attr:
        case "xpos":
            min, max = 0, 480
        case "ypos":
            min, max = 0, 320
        case "width":
            min, max = 0, 320
        case "height":
            min, max = 0, 480
        case "angle":
            min, max = 0, 90
        case "corners":
            min, max = 3, 8
        case "area":
            min, max = 0, 153600/4
        case "relative-area":
            min, max = 0, 1
        case "bb-area":
            min, max = 0, 153600/4
        case "bb-area-ratio":
            min, max = 0, 1
        case "wh-ratio":
            min, max = 0, 10 # weird
        case "circle-distance":
            min, max = 0, 1
        case "white-level":
            min, max = 0, 30
        case "black-level":
            min, max = 0, 30
        case "color-mean-l":
            min, max = 0, 100
        case "color-mean-a":
            min, max = -120, 120
        case "color-mean-b":
            min, max = -120, 120
        case "color-std-l":
            min, max = 0, 100
        case "color-std-a":
            min, max = -128, 127
        case "color-std-b":
            min, max = -128, 127
        
    return normalise_val_in_range(val, min, max)

In [103]:
from skimage import color
def hsv2lab(h, s, v):
    h = h / 360
    s = s / 100
    v = v / 100
    rgb = color.hsv2rgb([h, s, v])
    return color.rgb2lab(rgb)

def transform_data(scene, fname):
    new_scene = {
        "image_index": int(fname.split('_')[2].split('.')[0]),
        "image_filename": (fname.rsplit('.', 1)[0] + '.png').lower(),
    }
    new_objects = []
    for object in scene["objects"]:
        new_object ={}


        new_object["attributes"] = object
        new_object["description"] = {}

        tmp1 = new_object["attributes"]["color-mean"]
        tmp1 = hsv2lab(tmp1[0], tmp1[1], tmp1[2])
        tmp2 = new_object["attributes"]["color-std"]
        tmp2 = hsv2lab(tmp2[0], tmp2[1], tmp2[2])

        del new_object["attributes"]["color-mean"]
        del new_object["attributes"]["color-std"]

        
        new_object["attributes"]["color-mean-l"] = tmp1[0]
        new_object["attributes"]["color-mean-a"] = tmp1[1]
        new_object["attributes"]["color-mean-b"] = tmp1[2]

        new_object["attributes"]["color-std-l"] = tmp2[0]
        new_object["attributes"]["color-std-a"] = tmp2[1]
        new_object["attributes"]["color-std-b"] = tmp2[2]

        new_object["attributes"]["angle"] = -new_object["attributes"]["angle"]

        for key, val in new_object["attributes"].items():
            norm_val = normalise_val(key, val)
            new_object["attributes"][key] = round(norm_val, 5)
            

        new_objects.append(new_object)
    new_scene["objects"] = new_objects
    return new_scene

In [104]:
base_dir = os.path.join(os.getenv('HOME'), "Corpora/CLEVR-extracted/scenes/")
input_dir = os.path.join(base_dir, "val")
output_dir = os.path.join(base_dir, "t-val")

for root, dirs, files in os.walk(input_dir):
    for fname in files:
        # if fname is not .DS_Store
        if (not fname.startswith('.')):
            with open(os.path.join(root, fname)) as f:
                scene = json.load(f)
                scene = transform_data(scene, fname)
                
            with open(os.path.join(output_dir, fname.replace("CLEVR", "clevr")), "w") as f:
                json.dump(scene, f)
    